# Data Preparation

In this notebook we will prepare the Instacart dataset in the following way

- remove rare items with less than 10 transactions
- Split in training/validation/testing set with following criteria
    1. Users who have more than one transaction: their most recent transaction is used for testing
    2. Users who have more than two transactions: their second-to-last transaction is used for validation
    3. All others are used for training
    
Training Set: used to learn all embeddings and the recommendation model   
Validation Set: used for selecting the hyper parameters   
Testing Set: used for reporting the final results   

In [34]:
from collections import Counter

import numpy as np
import pandas as pd

pd.options.mode.chained_assignment = None  # default='warn'

In [35]:
min_item_transactions = 500
min_user_transactions = 0
subset_user = 0.1

In [36]:
order_products_train_df = pd.read_csv("data/order_products__train.csv")
order_products_prior_df = pd.read_csv("data/order_products__prior.csv")
order_products_df = pd.concat([order_products_train_df, order_products_prior_df])[['order_id', 'product_id']]
print(f"Dimensions of concatenated data frame: {order_products_df.shape} \n")
order_products_df.head()

Dimensions of concatenated data frame: (33819106, 2) 



,order_id,product_id
0,1,49302
1,1,11109
2,1,10246
3,1,49683
4,1,43633


In [37]:
orders_df = pd.read_csv("data/orders.csv")[['order_id', 'user_id', 'order_number', 'eval_set']]
# remove the orders belonging to the test set, as we don't have those transactions
orders_df = orders_df[orders_df.eval_set != "test"]
# Get the number of times a user has ordered (filtering users with less than min_user_transactions)
tmp = orders_df['user_id'].value_counts()
user_count = tmp.reset_index()
user_count.columns = ['user_id', 'count']
# Select a random subset of users if subset_user is specified (for rapid prototyping)
user_count['select'] = (np.random.rand(user_count.shape[0]) < subset_user)
orders_df = pd.merge(orders_df, user_count.loc[(user_count['select']) & (user_count['count']>=min_user_transactions)], on = 'user_id')
orders_df.head()

,order_id,user_id,order_number,eval_set,count,select
0,1737705,17,1,prior,41,True
1,1681401,17,2,prior,41,True
2,2680214,17,3,prior,41,True
3,3197376,17,4,prior,41,True
4,3237467,17,5,prior,41,True


In [38]:
# Load and merge products, departments, aisles
products = pd.read_csv("data/products.csv")
departments = pd.read_csv("data/departments.csv")
products = pd.merge(products, departments, on='department_id')
aisles = pd.read_csv("data/aisles.csv")
products = pd.merge(products, aisles, on='aisle_id')
products.head()

,product_id,product_name,aisle_id,department_id,department,aisle
0,1,Chocolate Sandwich Cookies,61,19,snacks,cookies cakes
1,78,Nutter Butter Cookie Bites Go-Pak,61,19,snacks,cookies cakes
2,102,Danish Butter Cookies,61,19,snacks,cookies cakes
3,172,Gluten Free All Natural Chocolate Chip Cookies,61,19,snacks,cookies cakes
4,285,Mini Nilla Wafers Munch Pack,61,19,snacks,cookies cakes


In [39]:
order_products_df = pd.merge(order_products_df, orders_df[['order_id']], on='order_id')
# Get the number of times a product has been ordered (filtering products with less than min_item_transactions)
tmp = order_products_df['product_id'].value_counts()
item_count = tmp.reset_index()
item_count.columns = ['product_id', 'count']
# create array of Item_count: # of times this count appears in dataset and sort by item_count
count_i = np.array(list(Counter(item_count["count"].values).items()))
count_i = count_i[np.argsort(count_i[:,0]), :]
# merge products df with items who have a count >= min_item_transactions
item_descr = pd.merge(products, item_count.loc[item_count["count"]>=min_item_transactions], on = 'product_id').sort_values(['count'], ascending=False)    
item_descr.head()

,product_id,product_name,aisle_id,department_id,department,aisle,count
1006,24852,Banana,24,4,produce,fresh fruits,49787
992,13176,Bag of Organic Bananas,24,4,produce,fresh fruits,40819
1001,21137,Organic Strawberries,24,4,produce,fresh fruits,28497
808,21903,Organic Baby Spinach,123,4,produce,packaged vegetables fruits,25794
1045,47209,Organic Hass Avocado,24,4,produce,fresh fruits,22816


In [40]:
# merge order_products df with product_ids and 
order_products_df = pd.merge(order_products_df, item_descr[['product_id']], on = 'product_id')
# map product_id to numbers from 0 - len(item_list)
item_list = item_descr['product_id'].values
item_dict = dict(zip(item_list, np.arange(len(item_list))))

In [41]:
# Mapping product id to numbers between 0 - len(item_list)
item_descr['product_id'] = item_descr['product_id'].apply(lambda x: item_dict[x])

In [42]:
# merge order_products_df with orders_df and 
orders_df_full = pd.merge(order_products_df, orders_df, on='order_id')
# map user_id to numbers from 0 - len(user_list)
user_list = np.array(list(set(orders_df_full['user_id'].values)))
user_dict = dict(zip(user_list, np.arange(len(user_list))))

In [43]:
# apply product mapping using item_dict
# turn sequence of products into list of products for each order for each user
orders_df_full = orders_df_full.groupby(['eval_set', 'user_id', 'order_number'])['product_id'].apply(lambda x: [item_dict[k] for k in x])
orders_df_full = orders_df_full.reset_index()

In [44]:
# apply user mapping using user_dict
orders_df_full['user_id'] = orders_df_full['user_id'].apply(lambda x : user_dict[x])
orders_df_full = orders_df_full.sort_values(['order_number'])

In [45]:
# Get the indexes for the test set (last transaction) and validation set (second-to-last) transaction
temp_users = orders_df_full.groupby('user_id')
test_set_index = []
validation_set_index = []
for (k, d) in temp_users:
    if len(d) > 1:
        test_set_index.append(d.index[-1])
    if len(d) > 2:
        validation_set_index.append(d.index[-2])
np.random.seed(0)

In [46]:
orders_df_full.loc[:, "eval_set"] = "train"
orders_df_full.loc[validation_set_index, "eval_set"] = "validation"
orders_df_full.loc[test_set_index, "eval_set"] = "test"

In [47]:
item_descr.to_csv("input_data/small_product_metadata.csv", index=False)
orders_df_full.to_csv("input_data/small_full_order_data.csv", index=False)

In [48]:
train_data = orders_df_full[['user_id', 'product_id']].loc[orders_df_full['eval_set'] == 'train']
validation_data = orders_df_full[['user_id', 'product_id']].loc[orders_df_full['eval_set'] == 'validation']
test_data = orders_df_full[['user_id', 'product_id']].loc[orders_df_full['eval_set'] == 'test']

In [49]:
print(f'Average Basket Size (train): {np.mean(train_data["product_id"].apply(lambda x: len(x)))}')
print(f'Average Basket Size (validation): {np.mean(validation_data["product_id"].apply(lambda x: len(x)))}')
print(f'Average Basket Size (test): {np.mean(test_data["product_id"].apply(lambda x: len(x)))}')

Average Basket Size (train): 6.239985339531373
Average Basket Size (validation): 6.018576779026217
Average Basket Size (test): 6.083129225041638
